In [1]:
from IPython.display import display

from sympy import *

In [2]:
# порядок разложения
n = 12

In [3]:
x, h, C = symbols('x, h, C', real=True)
f, F = symbols('f, F', cls=Function)
xs = symbols('x_i ' + ' '.join('x_i+%d' % i for i in range(1, n)))
fs = symbols(' '.join('f^(%d)' % i for i in range(n)))
ls = symbols(' '.join('ls%d' % i for i in range(n+1)), real=True)
rs = symbols(' '.join('rs%d' % i for i in range(n+1)), real=True)
Sf = sum(fs[i]*x**i/factorial(i) for i in range(n))
# C - константа интегрирования
SF = sum(fs[i]*x**(i+1)/factorial(i+1) for i in range(n)) + C

def error(g):
    g = expand(g)
    for i in range(n):
        if g.coeff(h, i):
            return g.coeff(h, i)*h**i

In [4]:
def NCFP(bound, lpts, rpts, order):
    """
Компактные формулы Ньютона-Котеса для дробных точек
bound - список из двух границ, могут быть дробными 
lpts  - список целых точек для значения первообразной F
rpts  - список целых точек для значения функции f
order - порядок разложения
ls_i, rs_i - произвольные параметры
    """
    assert bound[0] != bound[1]
    assert bound[0] not in lpts
    assert bound[1] not in lpts
    eq = expand(SF.subs(x, bound[1]*h) - SF.subs(x, bound[0]*h) -\
                  sum(ls[i]*SF.subs(x, i*h) for i in lpts) -\
                h*sum(rs[i]*Sf.subs(x, i*h) for i in rpts))
    eq0 = eq.coeff(C, 0)
    eq1 = eq.coeff(C, 1)
    eqs = [eq0.coeff(h, p) for p in range(order)]
    eqs.append(eq1)
    s = solve(eqs, [ls[i] for i in lpts] + [rs[i] for i in rpts])
    if not s:
        return "there is no solution for NCFP(%s, %s, %s, %s)" %\
                                        (bound, lpts, rpts, order)
    else:
        return Eq(Integral(f(x), (x, bound[0], bound[1])), \
                 (sum(ls[i]*F(xs[i]) for i in lpts).subs(s) if lpts else 0) + \
                h*sum(rs[i]*f(xs[i]) for i in rpts).subs(s) +\
                  error(eq.subs(s)))

In [5]:
# F - тут как первообразная, но всегда в целых точках
NCFP([Rational(1, 3), Rational(5, 3)], [0, 1, 2], [0, 1, 2], 5)

Eq(Integral(f(x), (x, 1/3, 5/3)), h**5*(-f^(4)*rs2/30 - 2*f^(4)/729) + h*(rs2*f(x_i) + rs2*f(x_i+2) + (4*rs2 + 20/27)*f(x_i+1)) + (8/27 - 3*rs2)*F(x_i+2) + (3*rs2 - 8/27)*F(x_i))

In [6]:
# может использоваться при посторении разностных схем
# в точках 1/2
NCFP([Rational(1, 2), Rational(3, 2)], [0, 1, 2], [0, 1, 2], 7)

Eq(Integral(f(x), (x, 1/2, 3/2)), f^(6)*h**7/35840 + h*(-3*f(x_i)/64 + 9*f(x_i+1)/16 - 3*f(x_i+2)/64) - 17*F(x_i)/64 + 17*F(x_i+2)/64)

In [7]:
# может использоваться при посторении разностных схем
# при интегрировании по контуру (поверхности) в точках -1/3 от границы
NCFP([Rational(1, 3), Rational(5, 3)], [], [0, 1, 2], 5)

Eq(Integral(f(x), (x, 1/3, 5/3)), -22*f^(4)*h**5/3645 + h*(8*f(x_i)/81 + 92*f(x_i+1)/81 + 8*f(x_i+2)/81))

https://en.wikipedia.org/wiki/Newton%E2%80%93Cotes_formulas
## Closed Newton–Cotes Formulae

### Trapezoid rule

In [8]:
NCFP([0, 1], [], [0, 1], 3)

Eq(Integral(f(x), (x, 0, 1)), -f^(2)*h**3/12 + h*(f(x_i)/2 + f(x_i+1)/2))

### Simpson's rule

In [9]:
NCFP([0, 2], [], [0, 1, 2], 5)

Eq(Integral(f(x), (x, 0, 2)), -f^(4)*h**5/90 + h*(f(x_i)/3 + 4*f(x_i+1)/3 + f(x_i+2)/3))

### Simpson's 3/8 rule

In [10]:
NCFP([0, 3], [], [0, 1, 2, 3], 5)

Eq(Integral(f(x), (x, 0, 3)), -3*f^(4)*h**5/80 + h*(3*f(x_i)/8 + 9*f(x_i+1)/8 + 9*f(x_i+2)/8 + 3*f(x_i+3)/8))

### Boole's rule

In [11]:
NCFP([0, 4], [], [0, 1, 2, 3, 4], 7)

Eq(Integral(f(x), (x, 0, 4)), -8*f^(6)*h**7/945 + h*(14*f(x_i)/45 + 64*f(x_i+1)/45 + 8*f(x_i+2)/15 + 64*f(x_i+3)/45 + 14*f(x_i+4)/45))

## Open Newton–Cotes Formulae

### Rectangle rule

In [12]:
NCFP([0, 2], [], [1], 3)

Eq(Integral(f(x), (x, 0, 2)), f^(2)*h**3/3 + 2*h*f(x_i+1))

### Trapezoid method

In [13]:
NCFP([0, 3], [], [1, 2], 3)

Eq(Integral(f(x), (x, 0, 3)), 3*f^(2)*h**3/4 + h*(3*f(x_i+1)/2 + 3*f(x_i+2)/2))

### Milne's rule

In [14]:
NCFP([0, 4], [], [1, 2, 3], 5)

Eq(Integral(f(x), (x, 0, 4)), 14*f^(4)*h**5/45 + h*(8*f(x_i+1)/3 - 4*f(x_i+2)/3 + 8*f(x_i+3)/3))

### No name

In [15]:
NCFP([0, 5], [], [1, 2, 3, 4], 5)

Eq(Integral(f(x), (x, 0, 5)), 95*f^(4)*h**5/144 + h*(55*f(x_i+1)/24 + 5*f(x_i+2)/24 + 5*f(x_i+3)/24 + 55*f(x_i+4)/24))

## Compact Closed Newton–Cotes Formulae

### Simpson's 3/8 rule

In [16]:
NCFP([0, 3], [1, 2], [0, 1, 2, 3], 5)

Eq(Integral(f(x), (x, 0, 3)), h**5*(-11*f^(4)*rs3/30 + f^(4)/10) + h*(rs3*f(x_i) + rs3*f(x_i+3) + (6 - 13*rs3)*f(x_i+1) + (6 - 13*rs3)*f(x_i+2)) + (9 - 24*rs3)*F(x_i+1) + (24*rs3 - 9)*F(x_i+2))

In [17]:
NCFP([0, 3], [1, 2], [0, 1, 2, 3], 5).subs(rs[3], Rational(9, 24))

Eq(Integral(f(x), (x, 0, 3)), -3*f^(4)*h**5/80 + h*(3*f(x_i)/8 + 9*f(x_i+1)/8 + 9*f(x_i+2)/8 + 3*f(x_i+3)/8))

In [18]:
NCFP([0, 3], [1, 2], [0, 1, 2, 3], 7)

Eq(Integral(f(x), (x, 0, 3)), -3*f^(6)*h**7/1540 + h*(3*f(x_i)/11 + 27*f(x_i+1)/11 + 27*f(x_i+2)/11 + 3*f(x_i+3)/11) + 27*F(x_i+1)/11 - 27*F(x_i+2)/11)

### Boole's rule

In [19]:
NCFP([0, 4], [1, 2, 3], [0, 1, 2, 3, 4], 7)

Eq(Integral(f(x), (x, 0, 4)), h**7*(-5*f^(6)*rs4/42 + f^(6)/35) + h*(rs4*f(x_i) + rs4*f(x_i+4) + (12 - 34*rs4)*f(x_i+1) + (12 - 34*rs4)*f(x_i+3) + (36 - 114*rs4)*f(x_i+2)) + (28 - 90*rs4)*F(x_i+1) + (90*rs4 - 28)*F(x_i+3))

In [20]:
NCFP([0, 4], [1, 2, 3], [0, 1, 2, 3, 4], 7).subs(rs[4], Rational(28, 90))

Eq(Integral(f(x), (x, 0, 4)), -8*f^(6)*h**7/945 + h*(14*f(x_i)/45 + 64*f(x_i+1)/45 + 8*f(x_i+2)/15 + 64*f(x_i+3)/45 + 14*f(x_i+4)/45))

In [21]:
NCFP([0, 4], [1, 2, 3], [0, 1, 2, 3, 4], 9)

Eq(Integral(f(x), (x, 0, 4)), -f^(8)*h**9/2625 + h*(6*f(x_i)/25 + 96*f(x_i+1)/25 + 216*f(x_i+2)/25 + 96*f(x_i+3)/25 + 6*f(x_i+4)/25) + 32*F(x_i+1)/5 - 32*F(x_i+3)/5)

## Compact Open Newton–Cotes Formulae

### Trapezoid method

In [22]:
NCFP([0, 3], [1, 2], [1, 2], 3)

Eq(Integral(f(x), (x, 0, 3)), h**3*(-f^(2)*rs2/6 + f^(2)) + h*(rs2*f(x_i+1) + rs2*f(x_i+2)) + (3 - 2*rs2)*F(x_i+2) + (2*rs2 - 3)*F(x_i+1))

In [23]:
NCFP([0, 3], [1, 2], [1, 2], 3).subs(rs[2], Rational(3, 2))

Eq(Integral(f(x), (x, 0, 3)), 3*f^(2)*h**3/4 + h*(3*f(x_i+1)/2 + 3*f(x_i+2)/2))

In [24]:
NCFP([0, 3], [1, 2], [1, 2], 5)

Eq(Integral(f(x), (x, 0, 3)), f^(4)*h**5/10 + h*(6*f(x_i+1) + 6*f(x_i+2)) + 9*F(x_i+1) - 9*F(x_i+2))

### Milne's rule

In [25]:
NCFP([0, 4], [1, 2, 3], [1, 2, 3], 5)

Eq(Integral(f(x), (x, 0, 4)), h**5*(-f^(4)*rs3/30 + 2*f^(4)/5) + h*(rs3*f(x_i+1) + rs3*f(x_i+3) + (4*rs3 - 12)*f(x_i+2)) + (8 - 3*rs3)*F(x_i+3) + (3*rs3 - 8)*F(x_i+1))

In [26]:
NCFP([0, 4], [1, 2, 3], [1, 2, 3], 5).subs(rs[3], Rational(8, 3))

Eq(Integral(f(x), (x, 0, 4)), 14*f^(4)*h**5/45 + h*(8*f(x_i+1)/3 - 4*f(x_i+2)/3 + 8*f(x_i+3)/3))

In [27]:
NCFP([0, 4], [1, 2, 3], [1, 2, 3], 7)

Eq(Integral(f(x), (x, 0, 4)), f^(6)*h**7/35 + h*(12*f(x_i+1) + 36*f(x_i+2) + 12*f(x_i+3)) + 28*F(x_i+1) - 28*F(x_i+3))

### No name

In [28]:
NCFP([0, 5], [1, 2, 3, 4], [1, 2, 3, 4], 5)

Eq(Integral(f(x), (x, 0, 5)), h**5*(-f^(4)*rs2/60 + f^(4)*rs3/30 - 3*f^(4)*rs4/20 + f^(4)) + h*(rs2*f(x_i+2) + rs3*f(x_i+3) + rs4*f(x_i+4) + (rs2/3 - rs3/3 + rs4)*f(x_i+1)) + (-rs2/2 - rs3 + 9*rs4/2 - 10)*F(x_i+3) + (-rs2/2 + 2*rs3 - 9*rs4/2 + 10)*F(x_i+2) + (rs2/18 - 2*rs3/9 - 13*rs4/6 + 5)*F(x_i+4) + (17*rs2/18 - 7*rs3/9 + 13*rs4/6 - 5)*F(x_i+1))

In [29]:
NCFP([0, 5], [1, 4], [1, 2, 3, 4], 5)

Eq(Integral(f(x), (x, 0, 5)), h**5*(-f^(4)*rs4/10 + 8*f^(4)/9) + h*(rs4*f(x_i+1) + rs4*f(x_i+4) + (3*rs4 - 20/3)*f(x_i+2) + (3*rs4 - 20/3)*f(x_i+3)) + (55/9 - 8*rs4/3)*F(x_i+4) + (8*rs4/3 - 55/9)*F(x_i+1))

In [30]:
NCFP([0, 5], [1, 4], [1, 2, 3, 4], 5).subs(rs[4], Rational(3*55, 8*9))

Eq(Integral(f(x), (x, 0, 5)), 95*f^(4)*h**5/144 + h*(55*f(x_i+1)/24 + 5*f(x_i+2)/24 + 5*f(x_i+3)/24 + 55*f(x_i+4)/24))

In [31]:
NCFP([0, 5], [2, 3], [1, 2, 3, 4], 5)

Eq(Integral(f(x), (x, 0, 5)), h**5*(-11*f^(4)*rs4/30 + 3*f^(4)/2) + h*(rs4*f(x_i+1) + rs4*f(x_i+4) + (30 - 13*rs4)*f(x_i+2) + (30 - 13*rs4)*f(x_i+3)) + (55 - 24*rs4)*F(x_i+2) + (24*rs4 - 55)*F(x_i+3))

In [32]:
NCFP([0, 5], [2, 3], [1, 2, 3, 4], 5).subs(rs[4], Rational(55, 24))

Eq(Integral(f(x), (x, 0, 5)), 95*f^(4)*h**5/144 + h*(55*f(x_i+1)/24 + 5*f(x_i+2)/24 + 5*f(x_i+3)/24 + 55*f(x_i+4)/24))

In [33]:
NCFP([0, 5], [1, 2, 3, 4], [1, 2, 3, 4], 9)

Eq(Integral(f(x), (x, 0, 5)), f^(8)*h**9/126 + h*(20*f(x_i+1) + 120*f(x_i+2) + 120*f(x_i+3) + 20*f(x_i+4)) + 175*F(x_i+1)/3 + 100*F(x_i+2) - 100*F(x_i+3) - 175*F(x_i+4)/3)